Scrapper

In [1]:
import time
import pandas as pd
import requests
import json

from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Lets start by analysing the website of the Premier League (https://www.premierleague.com/home). Here we want to go to results. Second step is to choose the First Team Option, and select 'Filter' to choose our desired season. 

To do this, lets start by selecting the write URL which is 'https://www.premierleague.com/results?team=FIRST'. Now we are sure we have choosen the right division.

We proceed to initiate the bot.Here we have one key purpose: to find all the matchids of the season. This represent each of the 380 matches. These will then help us acccess the data of each match.

In [2]:
url = 'https://www.premierleague.com/results?team=FIRST/'

# Select Wait Time and Season
wait_time = 1
season_year = "2019/20"

browser = webdriver.Chrome()
browser.maximize_window()
wait = WebDriverWait(browser, 60)


browser.get(url)

# Accept Cookies
xpath_cookies = '//div[@class="btn-primary cookies-notice-accept"]/span'
cookies = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_cookies)))
time.sleep(wait_time)
cookies.click()

# Close No to Racism Tab
xpath_racism = '//a[@id="advertClose"]'
racism = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_racism)))
time.sleep(wait_time)
racism.click()

# Open Season Tab
xpath_season = '//div[@data-dropdown-current="compSeasons"]'
season = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_season)))
time.sleep(wait_time)
season.click()

# Select Season
xpath_year = f'//li[@data-option-name="{season_year}"]'
year = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_year)))
time.sleep(5)
year.click()

# Scroll to end to get all data
time.sleep(wait_time)
browser.find_element_by_xpath('//body').send_keys(Keys.CONTROL+Keys.END)
time.sleep(wait_time)
browser.find_element_by_xpath('//body').send_keys(Keys.CONTROL+Keys.END)

# Collecting all Match Ids
matchids = []
xpath_matchid = "//div[contains(@class,'active')]/div/section[@class='fixtures']/div/ul/li/div[@class]"
matchids = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath_matchid)))
while len(matchids) < 380:
    time.sleep(wait_time)
    browser.find_element_by_xpath('//body').send_keys(Keys.CONTROL+Keys.END)
    matchids = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath_matchid)))
    if len(matchids) == 380:
        break 
    elif len(matchids) > 380:
        print("Warning. This season had more than 380 matches.")
        
ids = []

for id in matchids:
    ids.append(id.get_attribute("data-matchid"))
    
list_ids = pd.DataFrame(ids)  

browser.close()
browser.quit()

In [3]:
list_ids

,0
0,46975
1,46976
2,46977
3,46978
4,46979
...,...
375,46607
376,46608
377,46611
378,46610


Here comes an important question to answer: if we knew the match IDs are in order, why we did not just selected the first and last match ID of the season we wanted? Simple, because we want to easily select any season we want! With this method we are going to be able to recreate this experiment with any season that we wish. Now, of course, we cannot use this same method for collecting the rest of our data because it would take too long. In this way, if by any chance there are minimal changes in the number of ids for any reason, our code will remain valid. 

Now comes the actual data scrapping. Here I am going to use the requests package. The reason for...

In [4]:
def club_id(info):
    id_home = info["entity"]["teams"][0]["team"]["id"]
    id_away = info["entity"]["teams"][1]["team"]["id"]
    return id_home, id_away

def find_stat (home_away, stat_name):
    data_format = info["data"][str(club_id(info)[home_away])]['M']
    for num, stat in enumerate(data_format):
        if data_format[num]["name"] == stat_name:
            value = data_format[num]["value"]
            return data_format[num]["value"]
        
def club_name (home_away):
    abbr_home = info["entity"]["teams"][0]["team"]["club"]["shortName"]
    abbr_away = info["entity"]["teams"][1]["team"]["club"]["shortName"]
    return abbr_home, abbr_away

headers = {
    'authority': 'footballapi.pulselive.com',
    'sec-ch-ua': '^\\^',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'accept': '*/*',
    'origin': 'https://www.premierleague.com',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.premierleague.com/',
    'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
    'if-none-match': 'W/^\\^0bc0822daddc052089673dacc1542f161^\\^',
}

# General Info
club_id_home = []
club_id_away = []
club_name_home = []
club_name_away = []
            
# Data

formation_home = []
formation_away = []
total_pass_home = []
total_pass_away = []
goals_scored_home = []
goals_scored_away = []
goals_conceded_home = []
goals_conceded_away = []
red_card_home = []
rec_card_away = []

for num, id in enumerate(list_ids[0]):
    response = requests.get('https://footballapi.pulselive.com/football/stats/match/'+str(id), headers=headers)
    #print(num)
    info = json.loads(response.text)   
    
    for i in list(range(2)):
        stat = find_stat(i, "formation_used")
        if i == 0:
            formation_home.append(stat)
        else:
            formation_away.append(stat)
    
    for i in list(range(2)):
        stat = find_stat(i, "total_pass")
        if i == 0:
            total_pass_home.append(stat)
        else:
            total_pass_away.append(stat)
            
    for i in list(range(2)):
        stat = find_stat(i, "goals")
        if i == 0:
            goals_scored_home.append(stat)    
        else:
            goals_scored_away.append(stat)
            
    for i in list(range(2)):
        stat = find_stat(i, "goals_conceded")
        if i == 0:
            goals_conceded_home.append(stat)
        else:
            goals_conceded_away.append(stat)
            
    for i in list(range(2)):
        stat = find_stat(i, "total_red_card")
        if i == 0:
            red_card_home.append(stat)
        else:
            rec_card_away.append(stat)
            
    for i in list(range(2)):
        if i == 0:
            club_name_home.append(club_name(info)[i])
        else:
            club_name_away.append(club_name(info)[i])
            
    for i in list(range(2)):
        if i == 0:
            club_id_home.append(club_id(info)[i])
        else:
            club_id_away.append(club_id(info)[i])
            
    print(num)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
d = {"Match ID": ids,
     "Home ID": club_id_home,
     "Away ID": club_id_away,
     "Home Name": club_name_home,
     "Away Name": club_name_away,
     "Home Formation": formation_home, 
     "Away Formation": formation_away,
     "Home Goals": goals_scored_home, 
     "Away Goals": goals_scored_away,
     "Home Goals Conceded": goals_conceded_home,
     "Away Goals Conceded": goals_conceded_away,
     "Home Passes": total_pass_home,
     "Away Passes": total_pass_away,
     "Red Cards Home": red_card_home,
     "Red Cards Away": rec_card_away
    }
df = pd.DataFrame(data=d)

df

,Match ID,Home ID,Away ID,Home Name,Away Name,Home Formation,Away Formation,Home Goals,Away Goals,Home Goals Conceded,Away Goals Conceded,Home Passes,Away Passes,Red Cards Home,Red Cards Away
0,46975,1,33,Arsenal,Watford,4231.0,4222.0,3.0,2.0,2.0,3.0,442.0,398.0,NaN,NaN
1,46976,43,131,Burnley,Brighton,442.0,3412.0,1.0,2.0,2.0,1.0,378.0,334.0,NaN,NaN
2,46977,4,38,Chelsea,Wolves,343.0,343.0,2.0,NaN,NaN,2.0,629.0,370.0,NaN,NaN
3,46978,6,21,Crystal Palace,Spurs,442.0,4231.0,1.0,1.0,1.0,1.0,422.0,489.0,NaN,NaN
4,46979,7,127,Everton,Bournemouth,4231.0,442.0,1.0,3.0,3.0,1.0,687.0,282.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,46607,43,20,Burnley,Southampton,442.0,3421.0,3.0,NaN,NaN,3.0,332.0,393.0,NaN,NaN
376,46608,6,7,Crystal Palace,Everton,442.0,4231.0,NaN,NaN,NaN,NaN,268.0,500.0,NaN,1.0
377,46611,33,131,Watford,Brighton,4222.0,343.0,NaN,3.0,3.0,NaN,414.0,442.0,NaN,NaN
378,46610,21,2,Spurs,Aston Villa,4312.0,4141.0,3.0,1.0,1.0,3.0,587.0,256.0,NaN,NaN
